<a href="https://colab.research.google.com/github/ppolldai/2021_1226_Paul_week1/blob/main/2021_1226_Paul_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)
metadata.head()

ratings.head()
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())



In [225]:

#找出得到評價5分最多的商品

sort_train=ratings_trainings.sort_values(by=['reviewerID', 'overall'])
filter5 = (sort_train["overall"] == 5)
filter_5 =sort_train[filter5]

freq5=filter_5['asin'].value_counts() 

#取前10個當作推薦基礎標的
head10=freq5.head(10)

#首先針對先前有評論過5分商品的消費者推播該商品，若沒評論過則以TOP10的五分商品作為推薦

ratings_by_user={}

for test_user in users:

  countk=0
  recommend=[]

#step1 
  mask1 = filter_5["reviewerID"] == test_user
  comment = filter_5[(mask1)]


  if not comment.empty:
    recommend.append(comment.iloc[0]['asin'])
    countk=countk+1

#step2  
  if countk<10:
    for hot in head10.items():
      recommend.append(hot[0])
      countk=countk+1
      if countk==10:
        break

  ratings_by_user[test_user]=recommend    


In [226]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):

    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.08305084745762711